In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import pyedflib
import matplotlib.pyplot as plt
from os import listdir

In [ ]:
edfs = sorted([f for f in listdir('.') if f.endswith('EDF')])
mws = sorted([f for f in listdir('.') if 'HRV Analysis' in f])

In [ ]:
for edf in edfs:
    # Get file information
    name = edf.removesuffix('.EDF')
    f = pyedflib.EdfReader(edf)
    summary = pd.DataFrame(f.getSignalHeaders())
    duration = f.getFileDuration()
    start = dt.datetime.timestamp(f.getStartdatetime()) * 1000
    end = start + duration
    
    # Get ECG recording info
    ecg_i = summary.index[summary['label'] == 'ECG'][0]
    fs = f.getSampleFrequency(ecg_i)
    unit = f.getPhysicalDimension(ecg_i)
    
    # Create EDF data frame
    ecg = pd.DataFrame({f'{unit}': f.readSignal(ecg_i)})
    ecg_time = pd.DataFrame(np.arange(
        start, end, (1 / fs)), columns = ['timestamp'])
    ecg = pd.concat([ecg_time, ecg], axis = 1).set_index('timestamp')
    
    # Write summary and ECG data to CSV
    summary.to_csv(f'./{name}_Summary.csv', index = False)
    ecg.to_csv(f'./{name}_ECG.csv', index = True)
    
    # Close EDF file
    f.close()
    del f